In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("/Users/mac/Documents/PHUNGPX/knowledge_graph_searching/.env")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

MODEL_NAME = "gpt-4.1-mini"

if "NEO4J_URI" not in os.environ:
    os.environ["NEO4J_URI"] = getpass.getpass("Enter your Neo4j URI: ")

if "NEO4J_USERNAME" not in os.environ:
    os.environ["NEO4J_USERNAME"] = getpass.getpass("Enter your Neo4j username: ")

if "NEO4J_PASSWORD" not in os.environ:
    os.environ["NEO4J_PASSWORD"] = getpass.getpass("Enter your Neo4j password: ")

In [2]:
import pandas as pd
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

In [3]:
def load_excel(
    file_path: str,
    sheet_name: str,
    ignored_column_names: list[str] = None
) -> list[str]:
    if ignored_column_names is None:
        ignored_column_names = []

    df = pd.read_excel(file_path, sheet_name=sheet_name)

    row_infos = []
    for _, row in df.iterrows():
        text_parts = []

        column_id = 0
        for column in df.columns:
            if column in ignored_column_names:
                continue

            if pd.notna(row[column]) and str(row[column]).strip():
                column_id += 1
                text_parts.append(f"{column_id}. `{column}`: {row[column]}")

        full_text = "\n".join(text_parts)

        row_infos.append(full_text)

    return row_infos

In [4]:
documents = load_excel(
    file_path="/Users/mac/Documents/PHUNGPX/knowledge_graph_searching/data/durian_pest_and_disease_data.xlsx",
    sheet_name="(3) Diseases Information",
    ignored_column_names=["No.", "References"]
)

In [5]:
print(documents[1])

1. `English Name`: Pink Disease
2. `Vietnamese Name`: Bệnh nấm hồng
3. `Thailand Name`: โรคราสีชมพู (Rok Ra Si Chomphu)
4. `Scientific Name`: Erythricium salmonicolor (Berk. & Broome) Burds. [Teleomorph]. The anamorph is Necator salmonicolor Berk. & Broome. Identification is confirmed.
5. `Alternative Names`: Corticium salmonicolor (older synonym), Branch Canker, Stem Canker, Cobweb Disease, Penyakit Kulapuk Merah Jambu (Malay), Cendawan Angin (Malay)
6. `Description`: Pink Disease is a severe biotic fungal disease affecting the woody stems and branches of durian trees. Caused by the fungus Erythricium salmonicolor, it is one of the most significant branch and trunk diseases in durian, especially in high-rainfall tropical regions. The disease manifests as a crust-like fungal growth that girdles branches, leading to rapid dieback and, in severe cases on the main trunk of young trees, tree mortality. Its wide host range, including rubber, cacao, and coffee, makes it a persistent threat i

In [6]:
from disease_schema import node_types, relation_types, allowed_relationships

NODE_TYPES = [node_type["label"] for node_type in node_types]
RELATION_TYPES = [relation_type["label"] for relation_type in relation_types]
ALLOWED_RELATIONSHIPS = allowed_relationships

In [7]:
NODE_TYPES

['CROP',
 'VARIETY',
 'DISEASE',
 'CROP_PART',
 'SYMPTOM',
 'PATHOGEN',
 'CONDITION',
 'SEASONALITY',
 'LOCATION',
 'TREATMENT',
 'PREVENTION_METHOD',
 'SPREAD_METHOD',
 'RISK_FACTOR']

In [8]:
RELATION_TYPES

['HAS_VARIETY',
 'AFFECTED_BY',
 'SUSCEPTIBLE_TO',
 'HAS_CROP_PART',
 'HAS_SYMPTOM',
 'HAS_DISEASE',
 'CAUSED_BY',
 'PEAKS_DURING',
 'MANAGED_BY',
 'PREVENTED_BY',
 'SPREADS_VIA',
 'TRIGGERED_BY',
 'OCCURS_IN']

In [9]:
ALLOWED_RELATIONSHIPS

[('CROP', 'HAS_VARIETY', 'VARIETY'),
 ('CROP', 'AFFECTED_BY', 'DISEASE'),
 ('VARIETY', 'SUSCEPTIBLE_TO', 'DISEASE'),
 ('CROP', 'HAS_CROP_PART', 'CROP_PART'),
 ('CROP_PART', 'HAS_SYMPTOM', 'SYMPTOM'),
 ('CROP_PART', 'HAS_DISEASE', 'DISEASE'),
 ('DISEASE', 'CAUSED_BY', 'PATHOGEN'),
 ('DISEASE', 'CAUSED_BY', 'CONDITION'),
 ('DISEASE', 'PEAKS_DURING', 'SEASONALITY'),
 ('DISEASE', 'MANAGED_BY', 'TREATMENT'),
 ('DISEASE', 'PREVENTED_BY', 'PREVENTION_METHOD'),
 ('DISEASE', 'SPREADS_VIA', 'SPREAD_METHOD'),
 ('DISEASE', 'TRIGGERED_BY', 'RISK_FACTOR'),
 ('DISEASE', 'OCCURS_IN', 'LOCATION')]

In [10]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0)
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=NODE_TYPES,
    allowed_relationships=ALLOWED_RELATIONSHIPS,
)

In [11]:
from langchain_core.documents import Document

documents = [Document(page_content=document) for document in documents]

graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)

for graph_document in graph_documents:
    print(f"Nodes ({len(graph_document.nodes)}):{graph_document.nodes}")
    print(f"Relationships ({len(graph_document.relationships)}):{graph_document.relationships}")

Nodes (57):[Node(id='Phomopsis Leaf Blight', type='Disease', properties={}), Node(id='Phomopsis Durionis', type='Pathogen', properties={}), Node(id='Diaporthe Sp.', type='Pathogen', properties={}), Node(id='Leaves', type='Crop_part', properties={}), Node(id='Twigs', type='Crop_part', properties={}), Node(id='Small Circular Water-Soaked Lesions', type='Symptom', properties={}), Node(id='Yellowish-Brown To Greyish-Brown Spots With Dark Brown Or Purplish Border And Yellow Halo', type='Symptom', properties={}), Node(id='Large Irregular Necrotic Patches', type='Symptom', properties={}), Node(id='Black Flask-Shaped Pycnidia', type='Symptom', properties={}), Node(id='Cankers On Twigs', type='Symptom', properties={}), Node(id='Twig Dieback', type='Symptom', properties={}), Node(id='Hot Humid Conditions', type='Condition', properties={}), Node(id='High Humidity', type='Condition', properties={}), Node(id='Prolonged Leaf Wetness', type='Condition', properties={}), Node(id='Wounds', type='Conditi

In [ ]:
DELETE_EXISTING_DATA = True

if DELETE_EXISTING_DATA:
    graph.query("MATCH (n) DETACH DELETE n")

graph.add_graph_documents(graph_documents)

## Querying

In [2]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("/Users/mac/Documents/PHUNGPX/knowledge_graph_searching/.env")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

MODEL_NAME = "gpt-4.1-mini"

if "NEO4J_URI" not in os.environ:
    os.environ["NEO4J_URI"] = getpass.getpass("Enter your Neo4j URI: ")

if "NEO4J_USERNAME" not in os.environ:
    os.environ["NEO4J_USERNAME"] = getpass.getpass("Enter your Neo4j username: ")

if "NEO4J_PASSWORD" not in os.environ:
    os.environ["NEO4J_PASSWORD"] = getpass.getpass("Enter your Neo4j password: ")

In [3]:
import pandas as pd
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

In [4]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Disease {id: STRING}
Crop {id: STRING}
Pathogen {id: STRING}
Symptom {id: STRING}
Variety {id: STRING}
Seasonality {id: STRING}
Location {id: STRING}
Treatment {id: STRING}
Crop_part {id: STRING}
Condition {id: STRING}
Spread_method {id: STRING}
Risk_factor {id: STRING}
Prevention_method {id: STRING}
Relationship properties:

The relationships:
(:Disease)-[:MANAGED_BY]->(:Treatment)
(:Disease)-[:CAUSED_BY]->(:Condition)
(:Disease)-[:CAUSED_BY]->(:Pathogen)
(:Disease)-[:TRIGGERED_BY]->(:Risk_factor)
(:Disease)-[:SPREADS_VIA]->(:Spread_method)
(:Disease)-[:PEAKS_DURING]->(:Seasonality)
(:Disease)-[:OCCURS_IN]->(:Location)
(:Disease)-[:PREVENTED_BY]->(:Prevention_method)
(:Crop)-[:AFFECTED_BY]->(:Disease)
(:Crop)-[:HAS_CROP_PART]->(:Crop_part)
(:Crop)-[:HAS_VARIETY]->(:Variety)
(:Variety)-[:SUSCEPTIBLE_TO]->(:Disease)
(:Crop_part)-[:HAS_SYMPTOM]->(:Symptom)


In [5]:
enhanced_graph = Neo4jGraph(enhanced_schema=True)
print(enhanced_graph.schema)

Node properties:
- **Disease**
  - `id`: STRING Example: "Phomopsis Leaf Blight"
- **Crop**
  - `id`: STRING Available options: ['Durian']
- **Pathogen**
  - `id`: STRING Example: "Phomopsis Durionis"
- **Symptom**
  - `id`: STRING Example: "Small Circular Water-Soaked Lesions"
- **Variety**
  - `id`: STRING Example: "D24 (Sultan)"
- **Seasonality**
  - `id`: STRING Available options: ['Rainy Season', 'Monsoon Or Rainy Season', 'Monsoon Season', 'Dry Season', 'Dry Season And Transitional Periods With Dew Forma', 'Rainy Season May To November', 'Rainy/Monsoon Season', 'May To October In Thailand And Vietnam', 'Late-Year Monsoons In Malaysia', 'Monsoon Period']
- **Location**
  - `id`: STRING Example: "Thailand"
- **Treatment**
  - `id`: STRING Example: "Mancozeb"
- **Crop_part**
  - `id`: STRING Example: "Leaves"
- **Condition**
  - `id`: STRING Example: "Hot Humid Conditions"
- **Spread_method**
  - `id`: STRING Example: "Rain Splash"
- **Risk_factor**
  - `id`: STRING Example: "Dense 

In [6]:
from langchain_neo4j import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
)

In [6]:
response = chain.invoke({"query": "Bệnh nào ảnh hưởng tới nhiều giống cây sầu riêng nhất?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Crop {id: 'Durian'})-[:HAS_VARIETY]->(v:Variety)-[:SUSCEPTIBLE_TO]->(d:Disease)
RETURN d.id AS Disease, COUNT(v) AS VarietyCount
ORDER BY VarietyCount DESC
LIMIT 1

Full Context:
[{'Disease': 'Fusarium Wilt Of Durian', 'VarietyCount': 14}]

> Finished chain.


{'query': 'Bệnh nào ảnh hưởng tới nhiều giống cây sầu riêng nhất?',
 'result': 'Bệnh Fusarium Wilt Of Durian ảnh hưởng tới nhiều giống cây sầu riêng nhất với 14 giống.'}

In [7]:
response = chain.invoke({"query": "Mùa nào xuất hiện nhiều sâu bệnh nhất?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease)-[:PEAKS_DURING]->(s:Seasonality)
RETURN s.id, COUNT(d) AS disease_count
ORDER BY disease_count DESC
LIMIT 1

Full Context:
[{'s.id': 'Rainy Season', 'disease_count': 5}]

> Finished chain.


{'query': 'Mùa nào xuất hiện nhiều sâu bệnh nhất?',
 'result': 'Mùa mưa xuất hiện nhiều sâu bệnh nhất với 5 loại bệnh.'}

In [9]:
response = chain.invoke({"query": "Bệnh nào xuất hiện của nhiều hơn 2 mùa trong năm và đó là những mùa nào?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease)-[:PEAKS_DURING]->(s:Seasonality)
WITH d, collect(s.id) AS seasons
WHERE size(seasons) > 2
RETURN d.id AS Disease, seasons

Full Context:
[{'Disease': 'Durian Leaf Anthracnose', 'seasons': ['Rainy/Monsoon Season', 'May To October In Thailand And Vietnam', 'Late-Year Monsoons In Malaysia']}]

> Finished chain.


{'query': 'Bệnh nào xuất hiện của nhiều hơn 2 mùa trong năm và đó là những mùa nào?',
 'result': 'Bệnh Durian Leaf Anthracnose xuất hiện trong nhiều hơn 2 mùa trong năm, đó là mùa mưa/monsoon, từ tháng 5 đến tháng 10 ở Thái Lan và Việt Nam, và các đợt gió mùa cuối năm ở Malaysia.'}

In [16]:
response = chain.invoke({"query": "Mùa mưa thường xuất hiện bệnh nào?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Disease)-[:PEAKS_DURING]->(s:Seasonality {id: 'Monsoon Or Rainy Season'})
RETURN d.id

Full Context:
[{'d.id': 'Pink Disease'}]

> Finished chain.


{'query': 'Mùa mưa thường xuất hiện bệnh nào?',
 'result': 'Bệnh thường xuất hiện trong mùa mưa là bệnh Pink Disease.'}

In [7]:
response = chain.invoke({"query": "Bệnh nào xuất hiện ở nhiều bộ phận trên cây nhất, đó là những bộ phận nào?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Crop)-[:AFFECTED_BY]->(d:Disease), (c)-[:HAS_CROP_PART]->(cp:Crop_part)
RETURN d.id AS Disease, collect(DISTINCT cp.id) AS CropParts, count(DISTINCT cp) AS NumberOfParts
ORDER BY NumberOfParts DESC
LIMIT 1

Full Context:
[{'Disease': 'Phomopsis Leaf Blight', 'CropParts': ['Young Fruits', 'Flowers', 'Shoots', 'Branches', 'Stem', 'Trunk', 'Fruit', 'Stems/Trunk', 'Roots', 'Leaves'], 'NumberOfParts': 10}]

> Finished chain.


{'query': 'Bệnh nào xuất hiện ở nhiều bộ phận trên cây nhất, đó là những bộ phận nào?',
 'result': 'Bệnh Phomopsis Leaf Blight xuất hiện ở nhiều bộ phận trên cây nhất, đó là Young Fruits, Flowers, Shoots, Branches, Stem, Trunk, Fruit, Stems/Trunk, Roots, Leaves.'}

In [15]:
response = chain.invoke({"query": "Có những Bộ phận nào trên cây sầu riêng, và bộ phần nào có nhiều disease nhất?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Crop {id: 'Durian'})-[:HAS_CROP_PART]->(cp:Crop_part)
OPTIONAL MATCH (cp)-[:HAS_SYMPTOM]->(:Symptom)<-[:AFFECTED_BY]-(c)-[:AFFECTED_BY]->(d:Disease)
RETURN cp.id AS CropPart, COUNT(d) AS DiseaseCount
ORDER BY DiseaseCount DESC

Full Context:
[{'CropPart': 'Young Fruits', 'DiseaseCount': 0}, {'CropPart': 'Flowers', 'DiseaseCount': 0}, {'CropPart': 'Shoots', 'DiseaseCount': 0}, {'CropPart': 'Branches', 'DiseaseCount': 0}, {'CropPart': 'Stem', 'DiseaseCount': 0}, {'CropPart': 'Trunk', 'DiseaseCount': 0}, {'CropPart': 'Fruit', 'DiseaseCount': 0}, {'CropPart': 'Stems/Trunk', 'DiseaseCount': 0}, {'CropPart': 'Roots', 'DiseaseCount': 0}, {'CropPart': 'Leaves', 'DiseaseCount': 0}]

> Finished chain.


{'query': 'Có những Bộ phận nào trên cây sầu riêng, và bộ phần nào có nhiều disease nhất?',
 'result': 'Trên cây sầu riêng có các bộ phận: Trái non, Hoa, Chồi, Cành, Thân, Thân cây, Trái, Thân/Thân cây, Rễ, và Lá. Không có bộ phận nào có nhiều bệnh nhất vì tất cả đều có số lượng bệnh là 0.'}